In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D, MaxPooling2D, Dot, Input, TimeDistributed, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA

2023-09-04 20:59:34.130357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

'''
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws_train = []
raws_test = []
for ii in range(1, 10):
    subject = f'S{ii:03d}'
    files = glob.glob(f'../files/{subject}/*.edf')
    for i in [5, 9, 13, 3, 7, 11]:
        current_file = files[i]
        r = read_raw_edf(current_file, preload=True, stim_channel='auto')
        events, _ = mne.events_from_annotations(r)
        if i in [5, 9, 13]:
            new_labels_events = {1:'rest', 2:'T1', 3:'T2'} # action
        elif i in [3, 7, 11]:
            new_labels_events = {1:'rest', 2:'T3', 3:'T4'}
        new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
        r.set_annotations(new_annot)
        if ii < 4:
            raws_test.append(r)
        else:
            raws_train.append(r)
    
raws_train_obj = concatenate_raws(raws_train)
raw_test_obj = concatenate_raws(raws_test)

Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/fil

In [4]:
def preprocess_raw(raw):
    # "Fc5.","Fc6.","Fc3.","Fc4.","Fc1.","Fc2.","C5..","C6..","C3..","C4..","C1..","C2..","Cp5.","Cp6.","Cp3.","Cp4.","Cp1.","Cp2."
    # raw = raw.pick_channels(['Fc5.', 'Fc6.', 'Fc3.', 'Fc4.', 'Fc1.', 'Fc2.', 'C5..', 'C6..', 'C3..', 'C4..', 'C1..', 'C2..', 'Cp5.', 'Cp6.', 'Cp3.', 'Cp4.', 'Cp1.', 'Cp2.'])

    # filters
    notch_freq = 60
    raw.notch_filter(notch_freq, fir_design='firwin')

    low_cutoff = 8
    high_cutoff = 40
    raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

    events, event_dict = mne.events_from_annotations(raw)
    print(raw.info)
    print(event_dict)
    picks = mne.pick_types(raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')

    event_id = {'T1': 2, 'T2': 3, 'T3': 4, 'T4': 5}
    events, event_dict = mne.events_from_annotations(raw, event_id=event_id)
    tmin = -0.2  # Time before event in seconds
    tmax = 0.8  # Time after event in seconds
    epochs = mne.Epochs(raw, events, event_dict, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

    # raw, events, event_dict, picks, epochs
    return raw, events, event_dict, picks, epochs

In [5]:

raw_train = raws_train_obj.copy()
raw_test = raw_test_obj.copy()

raw_train, events_train, event_dict_train, picks_train, epochs_train = preprocess_raw(raw_train)
raw_test, events_test, event_dict_test, picks_test, epochs_test = preprocess_raw(raw_test)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1057 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 36 contiguous segments


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    1.8s finished


Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (1.656 sec)



...

Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 8.0 Hz
 lowpass: 40.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'T1': 1, 'T2': 2, 'T3': 3, 'T4': 4, 'rest': 5}
Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4']
Not setting metadata
480 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 480 events and 641 original time points ...
0 bad epochs dropped
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequen

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 18 contiguous segments
Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 8.0 Hz
 lowpass: 40.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'T1': 1, 'T2': 2, 'T3': 3, 'T4': 4, 'rest': 5}
Used Annotations descriptions: ['T1', 'T2', 'T3', 'T4']
Not setting metadata
255 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 255 events and 641 original time points ...
0 bad epochs dropped


In [6]:
X_train = epochs_train.get_data()
y_train = epochs_train.events[:, -1] - 2
y_train_cat = to_categorical(y_train)

In [7]:
X_test = epochs_test.get_data()
y_test = epochs_test.events[:, -1] - 2
y_test_cat = to_categorical(y_test)

In [107]:
# cnn
# n_channels = X_train.shape[1]
# input_window_size = X_train.shape[2]
# input_shape = (1, n_channels, input_window_size)
# X_train = X_train.reshape(X_train.shape[0], 1, n_channels, input_window_size)
# X_test = X_test.reshape(X_test.shape[0], 1, n_channels, input_window_size)
# print(X_train.shape, X_test.shape)

(2460, 1, 64, 161) (255, 1, 64, 161)


In [10]:
# lstm / cnn 1d

n_channels = X_train.shape[1]
input_window_size = X_train.shape[2]
input_shape = (n_channels, input_window_size)
# X_train = X_train.reshape(X_train.shape[0], 1, n_channels, input_window_size)
# X_test = X_test.reshape(X_test.shape[0], 1, n_channels, input_window_size)
print(X_train.shape, X_test.shape)

(480, 64, 641) (255, 64, 641)


In [11]:
model = Sequential()

model.add(Conv1D(filters=640, kernel_size=11, strides=1, activation='elu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Conv1D(filters=630, kernel_size=11, strides=1, activation='elu', padding='same'))

# model.add(MaxPooling1D(pool_size=3))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=210, kernel_size=50, strides=1, activation='elu', padding='same'))
model.add(SpatialDropout1D(0.5))
# model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(filters=64, kernel_size=100, strides=1, activation='elu', padding='same'))
# # normalize
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=22, kernel_size=200, strides=1, activation='elu', padding='same'))
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=8, kernel_size=300, strides=1, activation='elu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(SpatialDropout1D(0.5))

model.add(MaxPooling1D(pool_size=2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(MaxPooling1D(pool_size=2))
# flatten
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# 4 classes output
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train_cat, epochs=250, batch_size=10, validation_data=(X_test, y_test_cat), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_train, y_train_cat, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')
# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print model summary
# model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 54, 640)           4513280   
                                                                 
 conv1d_1 (Conv1D)           (None, 54, 630)           4435830   
                                                                 
 spatial_dropout1d (SpatialD  (None, 54, 630)          0         
 ropout1D)                                                       
                                                                 
 conv1d_2 (Conv1D)           (None, 54, 210)           6615210   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 54, 210)          0         
 lDropout1D)                                                     
                                                                 
 conv1d_3 (Conv1D)           (None, 54, 64)           

KeyboardInterrupt: 

In [115]:
cnn_model = Sequential()
# Block 1: Temporal Convolution
cnn_model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.5))

# Block 2: Spacial Convolution
cnn_model.add(Conv2D(int(n_channels/2), (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('elu'))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())

# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
# model.add(MaxPooling2D())
input_shape_lstm = (None, 8)
model = Sequential()
model.add(TimeDistributed(cnn_model, input_shape=(1, n_channels, input_window_size)))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(4, activation='softmax'))

ValueError: Exception encountered when calling layer "time_distributed_11" (type TimeDistributed).

Input 0 of layer "sequential_41" is incompatible with the layer: expected shape=(None, 1, 64, 161), found shape=(None, 64, 161)

Call arguments received by layer "time_distributed_11" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 1, 64, 161), dtype=float32)
  • training=None
  • mask=None

In [76]:

model = Sequential()
# Block 1: Temporal Convolution
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block 2: Spacial Convolution
model.add(Conv2D(int(n_channels/2), (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# # Block 3: Separable Convolution
# model.add(Conv2D(8, (1, 1), strides=(1, 1), padding='same', use_bias=False))
# model.add(Conv2D(16, (1, 1), strides=(1, 1), padding='same', use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation('elu'))
# # model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
# model.add(Dropout(0.25))
# model.add(Flatten())










# # Convolutional Layer 1
# model.add(Conv2D(25, (11, 1), strides=(1, 1), input_shape=input_shape, padding='same', activation='elu'))

# # Convolutional Layer 2
# model.add(Conv2D(25, (1, 2), strides=(1, 1), padding='same', activation='elu'))

# # Max Pooling Layer 1
# model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

# # Convolutional Layer 3
# model.add(Conv2D(50, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# # Max Pooling Layer 2
# model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

# # Convolutional Layer 4
# model.add(Conv2D(100, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# # Max Pooling Layer 3
# model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding='same'))

# # Convolutional Layer 5
# model.add(Conv2D(200, (11, 1), strides=(1, 1), padding='same', activation='elu'))

# # Max Pooling Layer 4
# model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding='same'))

# model.add(Dropout(0.25))
# model.add(Flatten())

# # Classifier
# model.add(Dense(4, activation='softmax'))

ValueError: Input 0 of layer "conv2d_40" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 64, 161)

In [70]:
# model = Sequential()
# model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Activation('elu'))
# model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Activation('elu'))
# model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Activation('elu'))

# # Block 2: Spacial Convolution
# model.add(Conv2D(16, (n_channels, 1), strides=(1, 1), padding='same', use_bias=False))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
# model.add(Dropout(0.25))

# # Block 3: Separable Convolution
# model.add(Conv2D(16, (1, 16), strides=(1, 1), padding='same', use_bias=False))
# model.add(Conv2D(8, (1, 1), strides=(1, 1), padding='valid', use_bias=False))
# model.add(BatchNormalization(momentum=0.01, epsilon=1e-3))
# model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 8), strides=(1, 8)))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(4, activation='softmax'))

In [83]:
# model = Sequential()

# # Add LSTM layer
# model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))

# # Add attention mechanism
# model.add(Dense(1, activation='tanh'))
# model.add(Activation('softmax'))
# model.add(Dot(axes=[1, 1]))

# # Add Flatten layer
# model.add(Flatten())

# # Add fully connected layers
# model.add(Dense(64, activation='elu'))
# model.add(Dense(32, activation='elu'))
# model.add(Dropout(0.25))
# model.add(Dense(4, activation='softmax'))

# input_layer = Input(shape=input_shape)
# lstm_output = LSTM(units=128, return_sequences=True)(input_layer)

# # Attention mechanism
# attention = Dense(1, activation='tanh')(lstm_output)
# attention = Activation('softmax')(attention)
# attention = Dot(axes=[1, 1])([lstm_output, attention])
# attention_output = Flatten()(attention)

# # Fully connected layers
# fc_output = Dense(64, activation='elu')(attention_output)
# fc_output = Dense(32, activation='elu')(fc_output)
# output_layer = Dense(4, activation='softmax')(fc_output)
# model = Model(inputs=input_layer, outputs=output_layer)



model = Sequential()
# Block 1: Temporal Convolution
model.add(Conv2D(8, (1, n_channels), strides=(1, 1), padding='same', use_bias=False, input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block 2: Spacial Convolution
model.add(Conv2D(int(n_channels/2), (1, n_channels), strides=(1, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('elu'))
# model.add(AveragePooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1, 4)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

ValueError: Input 0 of layer "conv2d_43" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 64, 161)

In [130]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [114]:
model.fit(X_train, y_train_cat, epochs=250, batch_size=10, validation_data=(X_test, y_test_cat), verbose=1, shuffle=False)
loss, accuracy = model.evaluate(X_train, y_train_cat, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

Epoch 1/50


ValueError: in user code:

    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/si

In [74]:
loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=1)
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')

8/8 [==============================] - 0s 29ms/step - loss: 1.3944 - accuracy: 0.2353
Accuracy: 0.23529411852359772
Loss: 1.394429326057434
